In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
# county_sales_csv = "resources/Cannabis_Sales_By_County.csv"
# census_pop_csv = "resources/Census_pop.csv"
# county_per_cap_income_csv = "resources/County Per capita income.csv"
# retail_licenses_csv = "resources/retail-licenses.csv"
retail_map_csv = "output/retail_Map.csv"
city_dispo_csv = "output/city_dispo_county_with_county.csv"
storeLat_csv = "output/storeLatLong.csv"
county_sales_2019_csv = "output/2019_county_dispo_data_extended.csv"

In [2]:
# county_sales_df = pd.read_csv(county_sales_csv, parse_dates = ["Month Filter"])
# census_pop_df = pd.read_csv(census_pop_csv)
retail_map_df = pd.read_csv(retail_map_csv)
city_dispo_df = pd.read_csv(city_dispo_csv)
# store_lat_df = pd.read_csv(storeLat_csv)
county_sales_2019_df = pd.read_csv(county_sales_2019_csv)
# county_sales_2019_df

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../static/data/cannabis.db', echo=True)
conn = engine.connect()

2021-02-08 17:23:15,774 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-08 17:23:15,777 INFO sqlalchemy.engine.base.Engine ()
2021-02-08 17:23:15,778 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-08 17:23:15,778 INFO sqlalchemy.engine.base.Engine ()


In [84]:
conn.execute("""
DROP TABLE retail_map;
""")
conn.execute("""
    CREATE TABLE IF NOT EXISTS retail_map (
        id integer, 
        dispensary_name text,
        postal_city text,
        county text,
        full_address text,
        long integer,
        lat integer,
        PRIMARY KEY(id)
    )
""")

2021-02-06 14:31:40,382 INFO sqlalchemy.engine.base.Engine 
DROP TABLE retail_map;

2021-02-06 14:31:40,383 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 14:31:40,385 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-06 14:31:40,386 INFO sqlalchemy.engine.base.Engine 
    CREATE TABLE IF NOT EXISTS retail_map (
        id integer, 
        dispensary_name text,
        postal_city text,
        county text,
        full_address text,
        long integer,
        lat integer,
        PRIMARY KEY(id)
    )

2021-02-06 14:31:40,386 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 14:31:40,388 INFO sqlalchemy.engine.base.Engine COMMIT


In [89]:
retail_map_df.rename(columns = {"index":"id"},inplace=True)
# retail_map_df

In [99]:
# data = engine.execute("SELECT * FROM retail_map")
# for record in data:
#     print(record)

In [85]:
retail_map_df.reset_index(inplace=True)

In [90]:
retail_map_df.to_sql("retail_map", conn, if_exists="append", index=False)

2021-02-06 14:33:49,603 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("retail_map")
2021-02-06 14:33:49,604 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 14:33:49,607 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-06 14:33:49,615 INFO sqlalchemy.engine.base.Engine INSERT INTO retail_map (id, dispensary_name, postal_city, county, full_address, long, lat) VALUES (?, ?, ?, ?, ?, ?, ?)
2021-02-06 14:33:49,616 INFO sqlalchemy.engine.base.Engine ((0, '420Ville', 'Huntington', 'Baker', '210 W. Washington St. , Huntington, Or', -117.27113770000001, 44.350944899999995), (1, 'Hotbox Farms', 'Huntington', 'Baker', '120 E Washington St , Huntington, Or', -117.2661845, 44.3511435), (2, 'Sumpter Nugget', 'Sumpter', 'Baker', '160 N. Mill Street, Suite #102 , Sumpter, Or', -118.2021633, 44.745706899999995), (3, 'The Coughie Pot', 'Sumpter', 'Baker', '363 B South Mill Street Po Box 74, Sumpter, Or', -118.2023311, 44.7430577), (4, "Beatnick'S Finest", 'Corvallis', 'Benton', '

In [83]:
retail_map_df

,dispensary_name,postal_city,county,full_address,long,lat
0,420Ville,Huntington,Baker,"210 W. Washington St. , Huntington, Or",-117.27,44.35
1,Hotbox Farms,Huntington,Baker,"120 E Washington St , Huntington, Or",-117.27,44.35
2,Sumpter Nugget,Sumpter,Baker,"160 N. Mill Street, Suite #102 , Sumpter, Or",-118.20,44.75
3,The Coughie Pot,Sumpter,Baker,"363 B South Mill Street Po Box 74, Sumpter, Or",-118.20,44.74
4,Beatnick'S Finest,Corvallis,Benton,"126 Sw Avery Ave , Corvallis, Or",-123.27,44.55
...,...,...,...,...,...,...
732,Canna Bros.,Newberg,Yamhill,"2316 E Portland Rd Suite C-2 , Newberg, Or",-122.96,45.30
733,Oasis Cannabis,Newberg,Yamhill,"308 Villa Road, Suite 115 , Newberg, Or",-122.96,45.30
734,Western Oregon Dispensary Inc,Newberg,Yamhill,"1013 N. Springbrook Rd. , Newberg, Or",-122.95,45.31
735,Canna Bros.,Sheridan,Yamhill,"1210 W Main St , Sheridan, Or",-123.42,45.10


In [57]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS city_dispo (
        city text PRIMARY KEY,
        twenty_pop integer,
        nineteen_pop integer,
        long numeric,
        lat numeric,
        dispensary_count integer,
        population_per_dispensary numeric,
        county text        
    )
""")

2021-02-06 13:46:08,556 INFO sqlalchemy.engine.base.Engine 
    CREATE TABLE IF NOT EXISTS city_dispo (
        city text PRIMARY KEY,
        twenty_pop integer,
        nineteen_pop integer,
        long numeric,
        lat numeric,
        dispensary_count integer,
        population_per_dispensary numeric,
        county text        
    )

2021-02-06 13:46:08,557 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 13:46:08,558 INFO sqlalchemy.engine.base.Engine COMMIT


In [73]:
city_dispo_df.to_sql("city_dispo", conn, if_exists="append", index=False)

2021-02-06 14:18:50,909 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("city_dispo")
2021-02-06 14:18:50,910 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 14:18:50,911 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-06 14:18:50,914 INFO sqlalchemy.engine.base.Engine INSERT INTO city_dispo (city, twenty_pop, nineteen_pop, long, lat, dispensary_count, population_per_dispensary, county) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-02-06 14:18:50,916 INFO sqlalchemy.engine.base.Engine (('Albany', 50158, 55338, -123.10592820000001, 44.6365107, 8, 6269.75, 'Linn'), ('Ashland', 20078, 21281, -122.7094767, 42.194575799999996, 5, 4015.6, 'Jackson'), ('Astoria', 9477, 10015, -123.8312534, 46.1878841, 8, 1184.625, 'Clatsop'), ('Aurora', 918, 1039, -122.7559267, 45.2309536, 1, 918.0, 'Marion'), ('Bandon', 3066, 3148, -124.40844799999999, 43.118997799999995, 3, 1022.0, 'Coos'), ('Beaverton', 89803, 99037, -122.8013771, 45.4888761, 13, 6907.9230769999995, 'Washington'), ('Bend', 76

In [103]:
data = engine.execute("SELECT * FROM sales_yoy")
for record in data:
    print(record)

2021-02-06 15:04:42,636 INFO sqlalchemy.engine.base.Engine SELECT * FROM sales_yoy
2021-02-06 15:04:42,637 INFO sqlalchemy.engine.base.Engine ()
('2019-01-01 00:00:00.000000', 54280992.31, '2020-01-01 00:00:00.000000', 68653599.9)
('2019-02-01 00:00:00.000000', 52773889.66, '2020-02-01 00:00:00.000000', 69788785.58)
('2019-03-01 00:00:00.000000', 62119814.93000001, '2020-03-01 00:00:00.000000', 84863480.47999999)
('2019-04-01 00:00:00.000000', 62369873.760000005, '2020-04-01 00:00:00.000000', 89747591.62)
('2019-05-01 00:00:00.000000', 64574747.52, '2020-05-01 00:00:00.000000', 103437574.34)
('2019-06-01 00:00:00.000000', 66593082.79000001, '2020-06-01 00:00:00.000000', 100490240.53)
('2019-07-01 00:00:00.000000', 73144531.4, '2020-07-01 00:00:00.000000', 106619102.02000001)
('2019-08-01 00:00:00.000000', 79371526.47, '2020-08-01 00:00:00.000000', 103897664.94000001)
('2019-09-01 00:00:00.000000', 71611294.57, '2020-09-01 00:00:00.000000', 97939284.21000001)
('2019-10-01 00:00:00.00000

In [12]:
city_dispo_df.columns

Index(['city', '2020_pop', '2019_pop', 'long', 'lat', 'dispensary_count',
       'population_per_dispensary', 'county'],
      dtype='object')

In [72]:
city_dispo_df = city_dispo_df.rename(columns ={"2020_pop":"twenty_pop","2019_pop":"nineteen_pop"})

In [5]:
conn.execute("""
DROP TABLE county_sales;
""")
conn.execute("""
    CREATE TABLE IF NOT EXISTS county_sales (
        county text PRIMARY KEY,
        personal_income numeric,
        population integer,
        per_capita_income integer,
        dispensary_count integer,
        population_per_dispensary integer,
        sales integer,
        sales_per_capita numeric,
        avg_sales_per_dispensary numeric,
        avg_sale_dispo_per_capita numeric,
        percent_of_sales_over_income numeric,
        lat numeric,
        long numeric
    )
""")

2021-02-08 17:23:46,994 INFO sqlalchemy.engine.base.Engine 
DROP TABLE county_sales;

2021-02-08 17:23:46,994 INFO sqlalchemy.engine.base.Engine ()
2021-02-08 17:23:47,003 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-08 17:23:47,004 INFO sqlalchemy.engine.base.Engine 
    CREATE TABLE IF NOT EXISTS county_sales (
        county text PRIMARY KEY,
        personal_income numeric,
        population integer,
        per_capita_income integer,
        dispensary_count integer,
        population_per_dispensary integer,
        sales integer,
        sales_per_capita numeric,
        avg_sales_per_dispensary numeric,
        avg_sale_dispo_per_capita numeric,
        percent_of_sales_over_income numeric,
        lat numeric,
        long numeric
    )

2021-02-08 17:23:47,004 INFO sqlalchemy.engine.base.Engine ()
2021-02-08 17:23:47,014 INFO sqlalchemy.engine.base.Engine COMMIT


In [9]:
county_sales_2019_df.to_sql("county_sales", conn, if_exists="append", index=False)

2021-02-08 17:25:37,259 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("county_sales")
2021-02-08 17:25:37,260 INFO sqlalchemy.engine.base.Engine ()
2021-02-08 17:25:37,261 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-08 17:25:37,263 INFO sqlalchemy.engine.base.Engine INSERT INTO county_sales (county, personal_income, population, per_capita_income, dispensary_count, population_per_dispensary, sales, sales_per_capita, avg_sales_per_dispensary, avg_sale_dispo_per_capita, percent_of_sales_over_income, lat, long) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2021-02-08 17:25:37,263 INFO sqlalchemy.engine.base.Engine (('Baker', 685166000, 16124, 42494, 4, 4031, 30174052.05, 1871.38, 7543513.01, 467.84, 4.4, 44.7652, -117.6769), ('Umatilla', 3268316000, 77950, 41928, 4, 19488, 11665990.12, 149.66, 2916497.53, 37.41, 0.36, 45.775, -118.7606), ('Malheur', 1057034000, 30571, 34576, 9, 3397, 20271094.57, 663.08, 2252343.84, 73.68, 1.92, 43.2172, -117.5792), ('Washingto

In [6]:
county_sales_2019_df.columns

Index(['county', 'personal_income', 'population', 'per_capita_income',
       'dispensary_count', 'population_per_dispenary', 'sales',
       'sales_per_capita', 'avg_sales_per_dispensary',
       'avg_sale_dispo_per_capita', 'percent_of_sales_over_income', 'lat',
       'long'],
      dtype='object')

In [8]:
county_sales_2019_df.rename(columns = {"population_per_dispenary":"population_per_dispensary"},inplace=True)
county_sales_2019_df

,county,personal_income,population,per_capita_income,dispensary_count,population_per_dispensary,sales,sales_per_capita,avg_sales_per_dispensary,avg_sale_dispo_per_capita,percent_of_sales_over_income,lat,long
0,Baker,685166000,16124,42494,4,4031,30174052.05,1871.38,7543513.01,467.84,4.40,44.77,-117.68
1,Umatilla,3268316000,77950,41928,4,19488,11665990.12,149.66,2916497.53,37.41,0.36,45.77,-118.76
2,Malheur,1057034000,30571,34576,9,3397,20271094.57,663.08,2252343.84,73.68,1.92,43.22,-117.58
3,Washington,38527576000,601592,64043,40,15040,79856029.61,132.74,1996400.74,3.32,0.21,45.55,-123.14
4,Deschutes,11159204000,197692,56447,30,6590,43897085.12,222.05,1463236.17,7.40,0.39,43.83,-121.26
5,Clackamas,25812808000,418187,61726,30,13940,41266150.45,98.68,1375538.35,3.29,0.16,45.20,-122.12
6,Klamath,2839572000,68238,41613,5,13648,6415072.51,94.01,1283014.50,18.80,0.23,42.70,-121.61
7,Yamhill,5086605000,107100,47494,12,8925,14714908.24,137.39,1226242.35,11.45,0.29,45.18,-123.24
8,Josephine,3810451000,87487,43554,11,7953,12331177.75,140.95,1121016.16,12.81,0.32,42.34,-123.46
9,Multnomah,49399774000,812855,60773,216,3763,227971779.30,280.46,1055424.90,1.30,0.46,45.51,-122.59


In [62]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS sales_yoy (
        nineteen_months date PRIMARY KEY,
        nineteen_sales numeric,
        twenty_months date,
        twenty_sales numeric
    )
""")

2021-02-06 13:49:48,768 INFO sqlalchemy.engine.base.Engine 
    CREATE TABLE IF NOT EXISTS sales_yoy (
        nineteen_months date PRIMARY KEY,
        nineteen_sales numeric,
        twenty_months date,
        twenty_sales numeric
    )

2021-02-06 13:49:48,769 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 13:49:48,770 INFO sqlalchemy.engine.base.Engine COMMIT


In [77]:
yoy_sales.to_sql("sales_yoy", conn, if_exists="append", index=False)

2021-02-06 14:25:07,903 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("sales_yoy")
2021-02-06 14:25:07,903 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 14:25:07,905 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-06 14:25:07,906 INFO sqlalchemy.engine.base.Engine INSERT INTO sales_yoy (nineteen_months, nineteen_sales, twenty_months, twenty_sales) VALUES (?, ?, ?, ?)
2021-02-06 14:25:07,907 INFO sqlalchemy.engine.base.Engine (('2019-01-01 00:00:00.000000', 54280992.31, '2020-01-01 00:00:00.000000', 68653599.9), ('2019-02-01 00:00:00.000000', 52773889.66, '2020-02-01 00:00:00.000000', 69788785.58), ('2019-03-01 00:00:00.000000', 62119814.93000001, '2020-03-01 00:00:00.000000', 84863480.47999999), ('2019-04-01 00:00:00.000000', 62369873.760000005, '2020-04-01 00:00:00.000000', 89747591.62), ('2019-05-01 00:00:00.000000', 64574747.52, '2020-05-01 00:00:00.000000', 103437574.34), ('2019-06-01 00:00:00.000000', 66593082.79000001, '2020-06-01 00:00:00.000000', 1004

In [75]:
yoy_sales

,2019_months,2019_sales,2020_months,2020_sales
0,2019-01-01,54280992.31,2020-01-01,68653599.90
1,2019-02-01,52773889.66,2020-02-01,69788785.58
2,2019-03-01,62119814.93,2020-03-01,84863480.48
3,2019-04-01,62369873.76,2020-04-01,89747591.62
4,2019-05-01,64574747.52,2020-05-01,103437574.34
5,2019-06-01,66593082.79,2020-06-01,100490240.53
6,2019-07-01,73144531.40,2020-07-01,106619102.02
7,2019-08-01,79371526.47,2020-08-01,103897664.94
8,2019-09-01,71611294.57,2020-09-01,97939284.21
9,2019-10-01,70857737.06,2020-10-01,99128982.03


In [76]:
yoy_sales = yoy_sales.rename(columns ={"2019_months":"nineteen_months", "2019_sales":"nineteen_sales","2020_months":"twenty_months","2020_sales":"twenty_sales"})
yoy_sales

,nineteen_months,nineteen_sales,twenty_months,twenty_sales
0,2019-01-01,54280992.31,2020-01-01,68653599.90
1,2019-02-01,52773889.66,2020-02-01,69788785.58
2,2019-03-01,62119814.93,2020-03-01,84863480.48
3,2019-04-01,62369873.76,2020-04-01,89747591.62
4,2019-05-01,64574747.52,2020-05-01,103437574.34
5,2019-06-01,66593082.79,2020-06-01,100490240.53
6,2019-07-01,73144531.40,2020-07-01,106619102.02
7,2019-08-01,79371526.47,2020-08-01,103897664.94
8,2019-09-01,71611294.57,2020-09-01,97939284.21
9,2019-10-01,70857737.06,2020-10-01,99128982.03


In [45]:
# oregon_city_pop = census_pop_df.loc[census_pop_df["Geographic Area"].str.endswith("Oregon"), ["Geographic Area", "Census", "2019"]].reset_index(drop=True)
# oregon_city_pop = oregon_city_pop.rename(columns={"Census": "2020"})
# oregon_city_pop

In [10]:
# for index, row in county_per_cap_income_df.iterrows():
#     county_name = row["County"].split(", ")[0]
#     county_per_cap_income_df.loc[index, "County"] = county_name
# county_per_cap_income_df

In [43]:
# county_per_cap_income_df.dtypes
# county_sales2019_df["Sales"] = county_sales2019_df["Sales"].astype("int64")
# county_sales2019_df = county_sales2019_df.loc[:, ["County","Sales"]]
# county_sales2019_gb = county_sales2019_df.groupby("County").sum()
# # county_sales2019_gb

In [44]:
# sales_per_capita_df = county_per_cap_income_df.merge(county_sales2019_gb, on="County", how="inner")
# sales_per_capita_df["Sales Per income"] = (sales_per_capita_df["Personal income (thousands of dollars)"] / sales_per_capita_df["Sales"])*100
# sales_per_capita_df["Sales Per Population"] = (sales_per_capita_df["Population"] / sales_per_capita_df["Sales"])*100
# sales_per_capita_df.sort_values("Sales Per Population", ascending = False)

In [11]:
# retail_licenses_df

In [ ]:
census_pop_df

In [24]:
county_sales_csv = "resources/Cannabis_Sales_By_County.csv"

In [25]:
county_sales_df = pd.read_csv(county_sales_csv, parse_dates = ["Month Filter"])

In [41]:
county_sales2019_df = county_sales_df.loc[county_sales_df["Month Filter"].dt.year == 2019, ["County", "Month Filter", "Sales"]]
# county_sales2019_df

In [40]:
total_2019_sales = county_sales2019_df.groupby("Month Filter").sum().reset_index(drop = False).rename(columns = {"Month Filter":"2019_months", "Sales": "2019_sales"})
# total_2019_sales

In [39]:
county_sales2020_df = county_sales_df.loc[county_sales_df["Month Filter"].dt.year == 2020, ["County", "Month Filter", "Sales"]]
# county_sales2020_df

In [38]:
total_2020_sales = county_sales2020_df.groupby("Month Filter").sum().reset_index(drop = False).rename(columns = {"Month Filter":"2020_months", "Sales": "2020_sales"})
# total_2020_sales

In [35]:
yoy_sales = pd.concat([total_2019_sales, total_2020_sales],axis=1,)
yoy_sales

,2019_months,2019_sales,2020_months,2020_sales
0,2019-01-01,54280992.31,2020-01-01,68653599.90
1,2019-02-01,52773889.66,2020-02-01,69788785.58
2,2019-03-01,62119814.93,2020-03-01,84863480.48
3,2019-04-01,62369873.76,2020-04-01,89747591.62
4,2019-05-01,64574747.52,2020-05-01,103437574.34
5,2019-06-01,66593082.79,2020-06-01,100490240.53
6,2019-07-01,73144531.40,2020-07-01,106619102.02
7,2019-08-01,79371526.47,2020-08-01,103897664.94
8,2019-09-01,71611294.57,2020-09-01,97939284.21
9,2019-10-01,70857737.06,2020-10-01,99128982.03


In [37]:
yoy_sales.to_csv("output/yoy_sales.csv")